In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv, ive
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_3D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta1 = np.arctan2(np.sqrt(x[2]**2 + x[1]**2), x[0])
    theta2 = np.arctan2(x[2], x[1])   
    
    return rt+ndt, (theta1, theta2)

In [3]:
def k(a, da, t, q, sigma=2):
    return 0.5 * (q - 0.5*sigma - da(t))

def psi(a, da, t, z, tau, q, sigma=2):
    kk = k(a, da, t, q, sigma)
    
    if 2*np.sqrt(a(t)*z)/(sigma*(t-tau))<=700:
        term1 = 1./(sigma*(t - tau)) * np.exp(- (a(t) + z)/(sigma*(t-tau)))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = iv(q/sigma-1, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * iv(q/sigma, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    else:
        term1 = 1./(sigma*(t - tau))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = ive(q/sigma-1, (a(t) + z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * ive(q/sigma, (a(t) + z)/(sigma*(t-tau)))
    
    return term1 * term2 * (term3 * term4 + term5)

def ie_bessel_fpt(a, da, q, z, sigma=2, dt=0.1, T_max=2):
    g = [0]
    T = [0]
    g.append(-2*psi(a, da, dt, z, 0, q, sigma))
    T.append(dt)
    
    for n in range(2, int(T_max/dt)+2):
        s = -2 * psi(a, da, n*dt, z, 0, q, sigma)

        for j in range(1, n):
            if a(j*dt) == 0:
                continue
            
            s += 2 * dt * g[j] * psi(a, da, n*dt, a(j*dt), j*dt, q, sigma)

        g.append(s)
        T.append(n*dt)
        
    g = np.asarray(g)
    T = np.asarray(T)
    
    gt = interp1d(T, g)
    return gt

In [4]:
def HSDM_3D_likelihood(prms, RT, Theta):
    a = lambda t: prms[0] - prms[1]*t
    a2 = lambda t: (a(t))**2
    da2 = lambda t: -2*prms[1] * a(t)
    
    ndt = prms[2]
    mu = np.array([prms[3], prms[4], prms[5]])
    
    T_max = min(max(RT), prms[0]/prms[1])
    fpt = ie_bessel_fpt(a2, da2, mu.shape[0], 0.000001, 
                        dt=0.02, T_max=T_max)
    
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.0001 and rt - ndt < T_max:
            mu_dot_x0 = mu[0]*np.cos(theta[0])
            mu_dot_x1 = mu[1]*np.sin(theta[0])*np.cos(theta[1]) 
            mu_dot_x2 = mu[2]*np.sin(theta[0])*np.sin(theta[1])
            term1 = a(rt - ndt) * (mu_dot_x0 + mu_dot_x1 + mu_dot_x2)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt-ndt)
            
            density = np.exp(term1 - term2) * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'b0_true': [],
               'b0_estimate': [],
               'lambda_true': [],
               'lambda_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': [],
               'mu3_true': [],
               'mu3_estimate': []}

min_b0 = 2
max_b0 = 5

min_lambda = .1
max_lambda = 2

min_ndt = 0.1
max_ndt = 1

min_mu = -3
max_mu = 3

In [6]:
for n in tqdm(range(33)):
    b0 = np.random.uniform(min_b0, max_b0)
    lamb = np.random.uniform(min_lambda, max_lambda)
    a = lambda t: b0 - lamb*t
    
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu)])
    
    RT = []
    Theta = []
    
    for i in range(500):
        rt, theta = simulate_HSDM_3D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = differential_evolution(HSDM_3D_likelihood,
                                     args=(RT, Theta), 
                                     bounds=[(min_b0, max_b0), (min_lambda, max_lambda),
                                             (min_ndt, max_ndt), (min_mu, max_mu),
                                             (min_mu, max_mu), (min_mu, max_mu)])
    
    min_ans = minimize(HSDM_3D_likelihood,
                       args=(RT, Theta),
                       method='nelder-mead',
                       x0=min_ans.x,
                       bounds=[(min_b0, max_b0), (min_lambda, max_lambda),
                               (min_ndt, max_ndt), (min_mu, max_mu),
                               (min_mu, max_mu), (min_mu, max_mu)])
    
    if min_ans.success:
        recovery_df['b0_true'].append(b0)
        recovery_df['lambda_true'].append(lamb)
        recovery_df['ndt_true'].append(ndt)
        recovery_df['mu1_true'].append(mu[0])
        recovery_df['mu2_true'].append(mu[1])
        recovery_df['mu3_true'].append(mu[2])

        recovery_df['b0_estimate'].append(min_ans.x[0])
        recovery_df['lambda_estimate'].append(min_ans.x[1])
        recovery_df['ndt_estimate'].append(min_ans.x[2])
        recovery_df['mu1_estimate'].append(min_ans.x[3])
        recovery_df['mu2_estimate'].append(min_ans.x[4])
        recovery_df['mu3_estimate'].append(min_ans.x[5])

100%|██████████████████████████████| 33/33 [1:59:41<00:00, 217.61s/it]


In [7]:
recovery_df = pd.DataFrame(recovery_df)
recovery_df.corr()

,b0_true,b0_estimate,lambda_true,lambda_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate
b0_true,1.000000,0.899918,-0.097682,-0.261523,-0.073811,-0.017286,-0.100465,-0.098765,0.352464,0.352129,0.117296,0.114647
b0_estimate,0.899918,1.000000,-0.208680,-0.185617,-0.144215,-0.172980,-0.275283,-0.275960,0.260084,0.269345,0.053198,0.050748
lambda_true,-0.097682,-0.208680,1.000000,0.899085,0.222808,0.253608,0.266670,0.264991,-0.334213,-0.344387,0.444276,0.446229
lambda_estimate,-0.261523,-0.185617,0.899085,1.000000,0.144441,0.101907,0.094332,0.090730,-0.403417,-0.404879,0.404279,0.405268
ndt_true,-0.073811,-0.144215,0.222808,0.144441,1.000000,0.976947,0.222654,0.223872,0.008401,0.006393,-0.103199,-0.101011
ndt_estimate,-0.017286,-0.172980,0.253608,0.101907,0.976947,1.000000,0.278148,0.279397,0.063471,0.056373,-0.068373,-0.067080
mu1_true,-0.100465,-0.275283,0.266670,0.094332,0.222654,0.278148,1.000000,0.999118,0.036868,0.028377,-0.027924,-0.024957
mu1_estimate,-0.098765,-0.275960,0.264991,0.090730,0.223872,0.279397,0.999118,1.000000,0.043452,0.035704,-0.045930,-0.042695
mu2_true,0.352464,0.260084,-0.334213,-0.403417,0.008401,0.063471,0.036868,0.043452,1.000000,0.998870,0.007203,0.002062
mu2_estimate,0.352129,0.269345,-0.344387,-0.404879,0.006393,0.056373,0.028377,0.035704,0.998870,1.000000,-0.000878,-0.005826


In [8]:
file_name = '_Recovery_data/IE_3d_recovery_linear_500_02.csv'
old_recovery_data = pd.read_csv(file_name, index_col=0)
recovery_df = pd.concat([old_recovery_data, 
                         recovery_df]).reset_index(drop=True)
recovery_df.to_csv(file_name)

In [9]:
recovery_df

,b0_true,b0_estimate,lambda_true,lambda_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate
0,2.148317,2.359235,1.438196,1.663758,0.505154,0.485704,-2.370349,-2.445583,-1.277261,-1.316184,-2.054767,-2.091872
1,3.635062,3.439260,0.473456,0.436974,0.528541,0.552092,-0.374021,-0.417492,-2.106163,-2.025223,2.898536,2.858162
2,4.826623,4.532367,1.973275,1.786851,0.667942,0.711530,2.267376,2.340277,-0.491910,-0.526753,0.350809,0.376698
3,4.199072,3.893374,1.939724,2.000000,0.546225,0.621834,-0.195950,-0.277326,2.985417,3.000000,-0.283851,-0.328201
4,2.353982,2.409488,1.162493,1.237505,0.934116,0.938632,1.565139,1.498704,0.429792,0.449948,2.135110,2.060236
...,...,...,...,...,...,...,...,...,...,...,...,...
145,4.205662,4.592250,0.693741,0.894447,0.455778,0.362533,0.134794,0.155475,-1.387029,-1.233624,1.815400,1.601631
146,4.538608,4.179017,0.817541,0.493727,0.643023,0.673688,2.548948,2.639880,1.629193,1.693385,1.901436,2.030054
147,2.736420,3.346869,0.415976,0.812540,0.400639,0.304024,-0.712502,-0.684572,-2.124820,-2.139587,-1.648612,-1.586087
148,3.168679,3.483636,1.660585,2.000000,0.904461,0.896824,2.989982,3.000000,-0.870292,-0.887883,1.958710,2.031857
